<a href="https://colab.research.google.com/github/GrizzlyToast/ML_Practise/blob/main/BabyCryClassifier(Wav2Vec).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Consider removing Age '22' beause too old

In [1]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [35]:
!kaggle datasets download -d mennaahmed23/baby-crying-sounds-dataset
#! unzip "infant-cry-audio-corpus.zip"
! unzip "baby-crying-sounds-dataset.zip"

Dataset URL: https://www.kaggle.com/datasets/mennaahmed23/baby-crying-sounds-dataset
License(s): unknown
baby-crying-sounds-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  baby-crying-sounds-dataset.zip
  inflating: Baby Crying Sounds/belly pain/357c.wav  
  inflating: Baby Crying Sounds/belly pain/358c.wav  
  inflating: Baby Crying Sounds/belly pain/359c.wav  
  inflating: Baby Crying Sounds/belly pain/360c.wav  
  inflating: Baby Crying Sounds/belly pain/361c.wav  
  inflating: Baby Crying Sounds/belly pain/362c.wav  
  inflating: Baby Crying Sounds/belly pain/363c.wav  
  inflating: Baby Crying Sounds/belly pain/364c.wav  
  inflating: Baby Crying Sounds/belly pain/365c.wav  
  inflating: Baby Crying Sounds/belly pain/366c.wav  
  inflating: Baby Crying Sounds/belly pain/367c.wav  
  inflating: Baby Crying Sounds/belly pain/368c.wav  
  inflating: Baby Crying Sounds/belly pain/369c.wav  
  inflating: Baby Crying Sounds/belly 

In [3]:
!pip install datasets torchaudio librosa soundfile evaluate

In [4]:
import pandas as pd
import numpy as np
from datasets import load_dataset, Audio, Dataset, ClassLabel
import librosa
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification, SequenceFeatureExtractor, TrainingArguments, Trainer
import torch
import evaluate

In [5]:
model_id = "facebook/wav2vec2-large-xlsr-53"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_id)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

In [36]:
data_dir = "baby_crying_sounds"
data = []

for label in os.listdir(data_dir):
    label_path = os.path.join(data_dir, label)
    if os.path.isdir(label_path):
        for file_name in os.listdir(label_path):
            if file_name.endswith(".wav"):
                file_path = os.path.join(label_path, file_name)
                y, sr = librosa.load(file_path, sr=16000)
                data.append({"path": file_path, "array": y, "sampling_rate": sr, "label": label})


In [37]:
# Optionally define class labels
labels = sorted(list({d['label'] for d in data}))  # unique sorted labels
class_label = ClassLabel(names=labels)

# Convert label strings to integers
for d in data:
    d['label'] = class_label.str2int(d['label'])

# Convert list to Hugging Face dataset
dataset = Dataset.from_list(data)

# Set the features correctly
dataset = dataset.cast_column("label", class_label)

Casting the dataset:   0%|          | 0/1197 [00:00<?, ? examples/s]

In [40]:
labels = dataset.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [41]:
def preprocess_function(examples):
    audio_arrays = examples["array"]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=16000,
        truncation=True
    )
    return inputs


In [42]:
encoded_dataset = dataset.map(
    preprocess_function,
    remove_columns=["path", "array", "sampling_rate"],  # remove raw audio columns
    batched=True
)

Map:   0%|          | 0/1197 [00:00<?, ? examples/s]

In [43]:
encoded_dataset = encoded_dataset.train_test_split(test_size=0.2)

In [44]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_id, num_labels=num_labels, label2id=label2id, id2label=id2label
)

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [46]:
training_args = TrainingArguments(
    output_dir="baby_cry_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    processing_class=feature_extractor,
    compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.070443,0.304167
2,2.072800,2.063293,0.304167
3,2.053900,2.057088,0.304167
4,2.033500,2.050464,0.304167
5,2.018800,2.047086,0.304167


TrainOutput(global_step=40, training_loss=2.0447404384613037, metrics={'train_runtime': 907.7917, 'train_samples_per_second': 5.271, 'train_steps_per_second': 0.044, 'total_flos': 1.4502141092736e+17, 'train_loss': 2.0447404384613037, 'epoch': 5.0})

In [47]:
best_checkpoint = trainer.state.best_model_checkpoint
print("Best checkpoint:", best_checkpoint)

Best checkpoint: baby_cry_model/checkpoint-8


In [48]:
from transformers import pipeline
audio_file = "/content/donateacry_corpus/discomfort/1309B82C-F146-46F0-A723-45345AFA6EA8-1430704008-1.0-f-48-dc.wav"
classifier = pipeline("audio-classification", model="./baby_cry_model/checkpoint-8/")
classifier(audio_file)

Device set to use cuda:0


[{'score': 0.13016444444656372, 'label': 'hungry'},
 {'score': 0.12971322238445282, 'label': 'silence'},
 {'score': 0.12811841070652008, 'label': 'tired'},
 {'score': 0.12618164718151093, 'label': 'laugh'},
 {'score': 0.12336438149213791, 'label': 'burping'},
 {'score': 0.12285012751817703, 'label': 'discomfort'},
 {'score': 0.12097731232643127, 'label': 'belly pain'},
 {'score': 0.11863045394420624, 'label': 'cold_hot'}]

In [49]:
audio_file = "/content/donateacry_corpus/burping/7E4B9C14-F955-4BED-9B03-7F3096A6CBFF-1430232208-1.0-f-26-bu.wav"
classifier = pipeline("audio-classification", model="./baby_cry_model/checkpoint-8/")
classifier(audio_file)

Device set to use cuda:0


[{'score': 0.13003304600715637, 'label': 'hungry'},
 {'score': 0.12969525158405304, 'label': 'silence'},
 {'score': 0.12810789048671722, 'label': 'tired'},
 {'score': 0.12625418603420258, 'label': 'laugh'},
 {'score': 0.12338285148143768, 'label': 'burping'},
 {'score': 0.12280035018920898, 'label': 'discomfort'},
 {'score': 0.12096040695905685, 'label': 'belly pain'},
 {'score': 0.11876595765352249, 'label': 'cold_hot'}]

In [50]:
audio_file = "/content/donateacry_corpus/tired/06c4cfa2-7fa6-4fda-91a1-ea186a4acc64-1430029246453-1.7-f-26-ti.wav"
classifier = pipeline("audio-classification", model="./baby_cry_model/checkpoint-8/")
classifier(audio_file)

Device set to use cuda:0


[{'score': 0.1300567090511322, 'label': 'hungry'},
 {'score': 0.129682719707489, 'label': 'silence'},
 {'score': 0.12816619873046875, 'label': 'tired'},
 {'score': 0.1262141466140747, 'label': 'laugh'},
 {'score': 0.12338408827781677, 'label': 'burping'},
 {'score': 0.12280388176441193, 'label': 'discomfort'},
 {'score': 0.1209745854139328, 'label': 'belly pain'},
 {'score': 0.11871763318777084, 'label': 'cold_hot'}]